In [5]:
from azure.identity import InteractiveBrowserCredential
from itertools import chain, repeat
import struct
import pyodbc
import pandas as pd
import requests


# Authenticate using browser (same as SSMS MFA)
credential = InteractiveBrowserCredential()
token = credential.get_token("https://database.windows.net//.default")

# Encode token for ODBC
token_bytes = bytes(token.token, "UTF-8")
encoded = bytes(chain.from_iterable(zip(token_bytes, repeat(0))))
packed_token = struct.pack("<i", len(encoded)) + encoded
attrs = {1256: packed_token}  # SQL_COPT_SS_ACCESS_TOKEN


import jwt  # PyJWT
#from azure.identity import InteractiveBrowserCredential

# Authenticate and get token for Microsoft Graph
#credential = InteractiveBrowserCredential()
graph_token = credential.get_token("https://graph.microsoft.com/.default")

# Decode JWT token to extract claims
decoded = jwt.decode(graph_token.token, options={"verify_signature": False})
upn = decoded.get("upn") or decoded.get("preferred_username")

print("Logged in user:", upn)


Logged in user: admin@shakti.work


{'error': {'code': 'InvalidAuthenticationToken', 'message': 'Access token validation failure. Invalid audience.', 'innerError': {'date': '2025-08-19T11:18:33', 'request-id': '993a591e-ae8d-46af-bd51-da40823ebe7e', 'client-request-id': '993a591e-ae8d-46af-bd51-da40823ebe7e'}}}


In [10]:
def connection():
    server = "vv2iwx4oxwaebpogsiees6rd6q-3y5wf3jqtzmelnjb6zdd2nks2i.datawarehouse.fabric.microsoft.com"  # Replace with your actual server name
    database = "Food Waste Management"

    connection_string = f"""
    Driver={{ODBC Driver 17 for SQL Server}};
    Server={server};
    Database={database};
    Encrypt=Yes;
    TrustServerCertificate=No;
    """
    return connection_string


In [11]:
conn = pyodbc.connect(connection(), attrs_before=attrs)
cursor = conn.cursor()

# Example query
cursor.execute("SELECT TOP 10 * FROM sys.tables")
for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()


('claims_data', 1381579960, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2025, 8, 15, 14, 58, 47, 850000), datetime.datetime(2025, 8, 16, 8, 16, 18, 630000), False, False, False, 0, None, 9, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None, False)
('providers_data', 1413580074, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2025, 8, 15, 15, 4, 49, 630000), datetime.datetime(2025, 8, 16, 9, 29, 17, 650000), False, False, False, 0, None, 7, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None, False)
('receivers_data', 1429580131, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2025, 8, 15, 15, 6, 43, 357000), 

In [4]:
conn = pyodbc.connect(connection_string, attrs_before=attrs)
cursor = conn.cursor()

# Example query
cursor.execute(" SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
rows = cursor.fetchall()
columns = [column[0] for column in cursor.description]
df = pd.DataFrame.from_records(rows, columns=columns)
cursor.close()
conn.close()

In [ ]:
conn = pyodbc.connect(connection_string, attrs_before=attrs)
cursor = conn.cursor()

#schema = "dbo"
for schema, table in table_info:
    # Example query
    cursor.execute(f" SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{schema}' AND TABLE_NAME = '{table}' ")
    for row in cursor.fetchall():
        print(row)

    cursor.close()
    conn.close()

NameError: name 'table_info' is not defined

In [13]:
conn = pyodbc.connect(connection(), attrs_before=attrs)
cursor = conn.cursor()

# Example query
query = "SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = 'dbo' AND TABLE_NAME in (SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE')"
cursor.execute(query)
rows = cursor.fetchall()
columns = [column[0] for column in cursor.description]
df = pd.DataFrame.from_records(rows, columns=columns)
print(df)
cursor.close()
conn.close()

            TABLE_NAME    COLUMN_NAME  DATA_TYPE
0          claims_data         Status    varchar
1          claims_data      Timestamp  datetime2
2          claims_data       Claim_ID        int
3          claims_data        Food_ID        int
4          claims_data    Receiver_ID        int
5   food_listings_data        Food_ID        int
6   food_listings_data      Food_Name    varchar
7   food_listings_data       Quantity        int
8   food_listings_data    Expiry_Date       date
9   food_listings_data    Provider_ID        int
10  food_listings_data  Provider_Type    varchar
11  food_listings_data       Location    varchar
12  food_listings_data      Food_Type    varchar
13  food_listings_data      Meal_Type    varchar
14      providers_data           Name    varchar
15      providers_data           Type    varchar
16      providers_data        Address    varchar
17      providers_data           City    varchar
18      providers_data        Contact    varchar
19      providers_da

In [14]:
conn = pyodbc.connect(connection(), attrs_before=attrs)
cursor = conn.cursor()

# Example query
query = "SELECT SYSTEM_USER AS CurrentUser;"
cursor.execute(query)
rows = cursor.fetchall()
columns = [column[0] for column in cursor.description]
df = pd.DataFrame.from_records(rows, columns=columns)
print(df)
cursor.close()
conn.close()

         CurrentUser
0  admin@shakti.work
